In [16]:
from astropy.time import Time
import numpy as np
import pandas as pd
import astropy.units as u

## Load data

In [17]:
fink_grb_data_path = "/user/julien.peloton/fink_grb/"

In [18]:
df = spark.read\
.option("basePath", fink_grb_data_path)\
.format("parquet")\
.load([fink_grb_data_path + "gcn_storage/raw"]).toPandas()
df = df.sort_values("triggerTimejd")

## Simple stats

In [19]:
last_day = df["triggerTimeUTC"].values[-1]
print("first gcn date: {}\nlast gcn date: {}".format(
    df["triggerTimeUTC"].values[0], 
    last_day
))

first gcn date: 2023-05-29T20:15:00.746000000
last gcn date: 2023-07-10T15:54:09.512000000


#### GCN of the last day

In [20]:
df[df["triggerTimeUTC"] == last_day]

,observatory,instrument,event,ivorn,triggerId,ra,dec,err_arcmin,ackTime,triggerTimejd,triggerTimeUTC,raw_event,year,month,day
156,LVK,H1_L1,gw,,S230710p,20.25,44.99388,4.394709e+07,2023-07-10 17:59:58.260889,2.460136e+06,2023-07-10 15:54:09.512,"{""alert_type"": ""PRELIMINARY"", ""time_created"": ...",2023,7,10


### Nb gcn / day

In [21]:
df["dayjd"] = df["triggerTimejd"].round()
res_gb = df.groupby("dayjd").count()["observatory"]
"{:.3f} ± {:.3f} gcn/day".format(res_gb.mean(), res_gb.std())

'11.074 ± 4.953 gcn/day'

### Nb gcn / observatory

In [22]:
df.groupby("observatory").count()

,instrument,event,ivorn,triggerId,ra,dec,err_arcmin,ackTime,triggerTimejd,triggerTimeUTC,raw_event,year,month,day,dayjd
observatory,,,,,,,,,,,,,,,
Fermi,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52
ICECUBE,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
INTEGRAL,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21
LVK,218,218,218,218,218,218,218,218,218,218,218,218,218,218,218
SWIFT,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3


### Nb gcn / day / observatory

In [8]:
observatories = df["observatory"].unique()

In [9]:
for obs in observatories:
    nb_day = []
    for day in df["dayjd"]:
        nb_day.append(len(df[(df["dayjd"] == day) & (df["observatory"] == obs)]))
    
    print("{}: {:.3f} ± {:.3f} gcn/day".format(obs, np.mean(nb_day), np.std(nb_day)))
    print()

LVK: 9.559 ± 3.510 gcn/day

Fermi: 2.204 ± 1.597 gcn/day

INTEGRAL: 1.043 ± 1.607 gcn/day

SWIFT: 0.127 ± 0.333 gcn/day

ICECUBE: 0.274 ± 0.571 gcn/day



### Nb gcn / instruments

In [10]:
for obs in observatories:
    df_tmp = df[df["observatory"] == obs]
    print("--- {} ---".format(obs))
    gb_col = "instrument"
    if obs == "ICECUBE":
        gb_col = "event"
    print(df_tmp.groupby(gb_col).count()["triggerId"].to_markdown())
    print("\n")

--- LVK ---
| instrument          |   triggerId |
|:--------------------|------------:|
| H1                  |           1 |
| H1_L1               |         215 |
| L1                  |           1 |
| MbtaL-Hon-clustered |           1 |


--- Fermi ---
| instrument   |   triggerId |
|:-------------|------------:|
| GBM          |          52 |


--- INTEGRAL ---
| instrument   |   triggerId |
|:-------------|------------:|
| Refined      |           1 |
| Wakeup       |           1 |
| Weak         |          19 |


--- SWIFT ---
| instrument   |   triggerId |
|:-------------|------------:|
| BAT          |           1 |
| XRT          |           2 |


--- ICECUBE ---
| event   |   triggerId |
|:--------|------------:|
| BRONZE  |           1 |
| Cascade |           2 |
| GOLD    |           2 |




### GCN latency

Warning: this is the latency between the trigger time and the last gcn emitted for each events. This is not the latency computed with the first gcn emitted just after the emission.

In [11]:
df["jdAckTime"] = Time(df["ackTime"].values).jd
df["latency"] = (df["jdAckTime"] - df["triggerTimejd"]) * 24

#### GCN latency global

In [12]:
"{:.3f} ± {:.3f} hours".format(df["latency"].mean(), df["latency"].std())

'11.762 ± 55.612 hours'

In [13]:
df.groupby("observatory").agg(
    latency_mean=("latency", np.mean),
    latency_std=("latency", np.std),
    latency_min=("latency", np.min),
    latency_max=("latency", np.max)
)

,latency_mean,latency_std,latency_min,latency_max
observatory,,,,
Fermi,9.382374,12.549360,4.000984,58.956907
ICECUBE,5.360771,1.920363,4.098092,8.683546
INTEGRAL,5.164508,5.321667,4.001161,28.390164
LVK,13.215549,64.792024,4.082593,911.909466
SWIFT,4.227208,0.192336,4.007949,4.367451


### Error area

#### error area global

In [14]:
err_squaredeg = (df["err_arcmin"].values * u.arcmin**2).to_value(u.deg**2)
"{:.3f} ± {:.3f} deg²".format(err_squaredeg.mean(), err_squaredeg.std())

'3434.482 ± 4175.675 deg²'

#### error area / observatory

In [15]:
df["err_squaredeg"] = err_squaredeg
df.groupby("observatory").agg(
    err_area_mean=("err_squaredeg", np.mean),
    err_area_std=("err_squaredeg", np.std),
    err_area_min=("err_squaredeg", np.min),
    err_area_max=("err_squaredeg", np.max)
)

,err_area_mean,err_area_std,err_area_min,err_area_max
observatory,,,,
Fermi,0.282598,2.415510e-01,1.666667e-02,0.833333
ICECUBE,0.034216,4.556499e-02,1.888889e-04,0.093102
INTEGRAL,0.000016,9.278895e-07,1.372222e-05,0.000017
LVK,4710.529087,4.241080e+03,4.364327e+01,26525.036363
SWIFT,0.000002,2.509180e-06,1.388889e-07,0.000005
